<a href="https://colab.research.google.com/github/shashwatguptaa/DataScience/blob/main/resnet_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU torchsummary

In [1]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
!git clone https://github.com/shashwatguptaa/DataScience.git

Cloning into 'DataScience'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 26 (delta 6), reused 21 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 87.70 KiB | 501.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.


In [9]:
!cp /content

cp: missing destination file operand after '/content'
Try 'cp --help' for more information.


In [ ]:
import os
from abc import ABC,abstractmethod
from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

from torchvision import transforms, models
from torchvision.datasets import ImageFolder

from torchsummary import summary

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,root_dir,transform=None,target_size=(224,224),max_image_per_class=1000):
    self.root_dir=root_dir
    self.transform=transform
    self.target_size=target_size
    self.max_image=max_image_per_class
    # self.classes=os.listdir(root_dir)
    selected_classes = ['cloudy', 'desert']
    self.classes = [cls for cls in os.listdir(root_dir) if cls in selected_classes]
    self.classes_indx={cls:indx for indx,cls in enumerate(self.classes)}
    self.samples=[]
    class_image_sizes=defaultdict(list)

    for classes in self.classes:
      class_path=os.path.join(root_dir,classes)
      img_files=os.listdir(class_path)
      if(len(img_files)>self.max_image):
          img_files=img_files[:self.max_image]
      for img_name in img_files:
        img_path=os.path.join(class_path,img_name)
        self.samples.append((img_path,self.classes_indx[classes]))

    for img_path, class_idx in self.samples:
      with Image.open(img_path) as img:
          width, height = img.size
          class_name = self.classes[class_idx]
          class_image_sizes[class_name].append((width, height))

    for class_name, sizes in class_image_sizes.items():
      print(f"\nClass: {class_name}")
      print(f"Total images: {len(sizes)}")
      size_counts = defaultdict(int)
      for size in sizes:
          size_counts[size] += 1
      for size, count in size_counts.items():
          print(f"  Size {size}: {count} images")

  def __len__(self):
    return len(self.samples)

  def __getitem__(self,idx):
    image_path,label=self.samples[idx]
    image=Image.open(image_path).convert('RGB')
    image=image.resize(self.target_size)

    if self.transform:
      image=self.transform(image)

    return image,label

def createDataLoader(dataset,batch_size=32,train_size=0.7,val_size=0.2,test_size=0.1):
    total_size=len(dataset)
    train_size=int(train_size*total_size)
    val_size=int(val_size*total_size)
    test_size=total_size-train_size-val_size

    train_dataset,val_dataset,test_dataset=random_split(dataset,[train_size,val_size,test_size])

    train_dataset=DataLoader(train_dataset,batch_size,shuffle=True)
    val_dataset=DataLoader(val_dataset,batch_size,shuffle=False)
    test_dataset=DataLoader(test_dataset,batch_size,shuffle=False)

    return train_dataset,val_dataset,test_dataset

In [ ]:
class BaseModel(ABC,nn.Module):
  def __init__(self, num_classes):
      super().__init__()
      self.num_classes = num_classes

  @abstractmethod
  def forward(self,x):
      pass

  def get_optimizer(self,lr=0.001,optimizer_type='adam'):
    if(optimizer_type.lower()=='adam'):
      return optim.Adam(self.parameters(),lr=lr)
    elif(optimizer_type.lower()=='sgd'):
      return optim.SGD(self.parameters(),lr=lr)
    else:
      raise ValueError("Optimizer not supported")

  def get_criterion(self):
    return nn.CrossEntropyLoss()

In [ ]:
class CustomResnet(BaseModel):
  def __init__(self,num_features,pretrained=True,freeze_backbone=False):
    super(CustomResnet,self).__init__(num_features)
    self.resnet=models.resnet18(pretrained=pretrained)

    # if freeze_backbone:
    #   for param in self.resnet.parameters():
    #     param.requires_grad= False

    self.resnet.fc=nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(self.resnet.fc.in_features,512),
        nn.ReLU(inplace=True),
        nn.Dropout(0.3),
        nn.Linear(512,num_features)
    )

    # if freeze_backbone:
    #     for name, param in self.resnet.named_parameters():
    #         if not name.startswith('fc'):
    #             param.requires_grad = False

  def forward(self,x):
    return self.resnet(x)

In [ ]:
class CustomVgg(BaseModel):
  def __init__(self,num_features,pretrained=True,freeze_backbone=False):
    super(CustomVgg,self).__init__(num_features)
    self.vgg=models.vgg16(pretrained=pretrained)

    # if freeze_backbone:
    #   for param in self.vgg.parameters():
    #     param.requires_grad= False

    self.vgg.classifier=nn.Sequential(
        nn.Linear(25088,4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.3),
        nn.Linear(4096,1024),
        nn.ReLU(inplace=True),
        nn.Dropout(0.3),
        nn.Linear(1024,num_features)
    )

  def forward(self, x):
    return self.vgg(x)

In [ ]:
class CustomCNN(BaseModel):
  def __init__(self,num_features,input_channels=3):
    super(CustomCNN,self).__init__(num_features)
    self.input_channels=input_channels
    self.features=nn.Sequential(
        nn.Conv2d(input_channels,64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.AdaptiveAvgPool2d((7,7))
    )

    self.classifier=nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(512*7*7,4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.3),
        nn.Linear(4096,1024),
        nn.ReLU(inplace=True),
        nn.Dropout(0.3),
        nn.Linear(1024,num_features)
    )

  def forward(self,x):
    x=self.features(x)
    x=torch.flatten(x, 1)
    x=self.classifier(x)
    return x

In [ ]:
class ModelFactory:
  @staticmethod
  def create_model(model_name,num_features,**kwargs):
    if(model_name.lower() == 'resnet'):
      return CustomResnet(num_features,**kwargs)
    elif(model_name.lower() == 'vgg'):
      return CustomVgg(num_features,**kwargs)
    elif(model_name.lower() == 'cnn'):
      return CustomCNN(num_features,**kwargs)
    else:
      raise ValueError('This model is not defined.')

class Trainer:
  def __init__(self,model,train_loader,val_loader,device='cuda'):
    self.model=model.to(device)
    self.train_loader=train_loader
    self.val_loader=val_loader
    self.device=device
    self.optimiser=model.get_optimizer()
    self.criterion=model.get_criterion()

    self.train_losses = []
    self.val_losses = []
    self.train_accuracies = []
    self.val_accuracies = []


  def train_epoch(self):
    self.model.train()
    running_loss=0.0
    correct=0
    total=0
    for inputs,label in self.train_loader:
      inputs=inputs.to(self.device)
      label = label.to(self.device)

      self.optimiser.zero_grad()
      output=self.model(inputs)
      # print(output, ": The output hehehehe\n")
      loss=self.criterion(output,label)
      self.optimiser.step()

      running_loss+=loss.item()
      _,predicted=torch.max(output.data,1)
      total+=label.size(0)
      correct+=(predicted == label).sum().item()
    return running_loss/len(self.train_loader),100*correct/total

  def validate(self):
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in self.val_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)

                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        return running_loss / len(self.val_loader), 100 * correct / total

  def train(self, epochs,save_path='best_model.pth'):
        best_train_acc=0.0
        best_val_acc = 0.0
        for epoch in range(epochs):
            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc = self.validate()

            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_accuracies.append(train_acc)
            self.val_accuracies.append(val_acc)

            # if val_acc > best_val_acc:
            #   best_val_acc = val_acc
            #   torch.save(self.model.state_dict(), save_path)
            #   print(f"Best model saved with val acc: {val_acc:.2f}%")

            print(f'Epoch [{epoch+1}/{epochs}]')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
            print('-' * 50)

In [ ]:
def plot_training_curves(trainer):
    epochs = range(1, len(trainer.train_losses) + 1)

    # Loss plot
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, trainer.train_losses, label='Train Loss')
    plt.plot(epochs, trainer.val_losses, label='Val Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.legend()

    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, trainer.train_accuracies, label='Train Acc')
    plt.plot(epochs, trainer.val_accuracies, label='Val Acc')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy Curve')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
transform = transforms.Compose([
    # transforms.Resize((64, 64)),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = CustomDataset(
    root_dir="/content/drive/MyDrive/Untitled Folder/unzipped_data/data",
    transform=transform
)

train_loader, val_loader, test_loader = createDataLoader(dataset, batch_size=32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Class: cloudy
Total images: 1000
  Size (256, 256): 1000 images

Class: desert
Total images: 1000
  Size (256, 256): 1000 images


In [ ]:
resnet_model = ModelFactory.create_model('resnet', num_features=2, pretrained=True, freeze_backbone=True)
resnet_model.to(device)
summary(resnet_model, input_size=(3, 224, 224))
vgg_model = ModelFactory.create_model('vgg', num_features=2, pretrained=True, freeze_backbone=True)
vgg_model.to(device)
summary(vgg_model, input_size=(3, 224, 224))
custom_model = ModelFactory.create_model('cnn', num_features=2)

resnet_trainer = Trainer(resnet_model, train_loader, val_loader, device)
vgg_trainer = Trainer(vgg_model, train_loader, val_loader, device)
custom_trainer = Trainer(custom_model, train_loader, val_loader, device)

print("Training ResNet:")
resnet_trainer.train(epochs=20)

print("\nTraining VGG:")
vgg_trainer.train(epochs=20)

print("\nTraining Custom CNN:")
custom_trainer.train(epochs=20)


plot_training_curves(resnet_trainer)
plot_training_curves(vgg_trainer)
plot_training_curves(custom_trainer)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 141MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 126MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,